# 📰 NewsBot: AI-Powered News Digest with Gemma

Welcome to the **NewsBot** Notebook!

> In today's fast-paced world, staying informed without being overwhelmed is a constant challenge. This notebook provides a hands-on solution by demonstrating how to build a fully automated system for generating a concise, insightful news digest.

We will harness the power of **Gemma** 🧠, Google's family of lightweight, state-of-the-art open models, to perform sophisticated Natural Language Processing (NLP) tasks. The entire pipeline, from data fetching to final report, is handled right within this environment.

---


### 🛠️ Core Technologies of the first section

This project integrates several powerful tools to create the final digest:

* **Data Source:** A dedicated **News API** connection 🔗 to fetch the latest English-language articles focused on "Artificial Intelligence."
* **NLP Model:** **Gemma**, deployed via the **KerasNLP** library, for performing abstractive summarization.
* **Visualization:** Such as The **WordCloud**library 🖼️ to generate visual representations of key news trends.
* **Final Output:** A polished and shareable **HTML report** 📧.

---

### 🚀 The Digest Pipeline: From Raw Data to Final Report

This notebook guides you through each stage of the automated pipeline:

1.  **🌐 Data Acquisition**
    * We'll connect to the News API and pull a fresh batch of articles related to **AI**.

2.  **🧠 Intelligent Summarization**
    * You will use the Gemma model to condense lengthy articles into clear, digestible summaries. This process can be thought of as a function `$S(A) = s$`, where the model takes a full article `$A$` and produces a concise summary `$s$`.

3.  **📊 Insight & Visualization**
    * We'll analyze the summarized text to extract key themes and generate a compelling such as a **Word Cloud**, turning dense information into immediate visual insight.

4.  **✅ Report Generation**
    * Finally, all the generated content—summaries, trends, and the word cloud—will be compiled into a single, polished **HTML news digest**.

Get ready to transform your news consumption from a time-sink into a source of focused, **AI-driven insight**! 💡

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q -U keras
!pip install --upgrade keras-nlp
!pip install --upgrade tensorflow

In [ ]:
!pip uninstall -y tensorflow-text
!pip install tensorflow-text

In [ ]:
!pip uninstall -y tensorflow-text

# Downgrade TensorFlow and its addons to 2.18.0
!pip install --upgrade --force-reinstall \
    tensorflow==2.18.0 \
    tensorboard==2.18.0 \
    tensorflow-text==2.18.0

In [ ]:
# Downgrade or reinstall gymnasium to the exact version required
!pip install --upgrade --force-reinstall gymnasium==0.29.0

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [ ]:
import requests
import time

In [ ]:
from kaggle_secrets import UserSecretsClient

NEWS_API_KEY = UserSecretsClient().get_secret("NEWS_API_KEY")

In [ ]:
# Step 1: Define Endpoint
# url = f"https://newsapi.org/v2/top-headlines?country=us&apiKey={NEWS_API_KEY}"
# url = f"https://newsapi.org/v2/top-headlines?country=us&category=technology&apiKey={NEWS_API_KEY}"
url = f"https://newsapi.org/v2/everything?q=AI&language=en&apiKey={NEWS_API_KEY}"

def fetch_news():
    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json()
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} – fetched {len(data['articles'])} articles")
    return data

# always fetch on notebook start
news_data = fetch_news()

In [ ]:
import pandas as pd
from IPython.display import display, HTML # <-- Import for richer display
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Assuming 'url' and 'requests' are defined/imported correctly in previous cells.

# Step 2: Fetch News Data
response = requests.get(url)
response.raise_for_status() # Good practice to check for errors
news_data = response.json()

# --- NEW CODE FOR BETTER PROCESSING AND DISPLAY ---

# Step 3: Process News Articles
articles = news_data.get("articles", [])
df = pd.DataFrame(articles)

# Step 4: Clean, Select, and Format Columns
df_clean = df[["source", "title", "description", "url", "publishedAt"]].copy()

# Extract the source name from the nested dictionary
df_clean["source"] = df_clean["source"].apply(lambda x: x["name"] if isinstance(x, dict) else x)

# Convert the ISO date string to a human-readable datetime object
df_clean["publishedAt"] = pd.to_datetime(df_clean["publishedAt"]).dt.strftime('%Y-%m-%d %H:%M')

# Rename columns for presentation
df_clean.columns = ["Source", "Title", "Description", "URL", "Published At"]

# Step 5: Display Prettier Output (using display() and HTML styling)
print("📰 Latest News Digest 📰")
print("------------------------")

# Use 'styler' to make the DataFrame look better in the notebook
display(
    df_clean.head(10).style
    .set_properties(**{'font-size': '10pt', 'border': '1px solid lightgrey'})
    .set_table_styles([{'selector': 'th', 'props': [('background-color', '#f0f0f0')]}]),
)

# Optional: Save as CSV (if needed)
df_clean.to_csv("news_data.csv", index=False)

# Step 6: Visualization (Word Cloud of Titles)
print("\n☁️ Article Title Word Cloud ☁️")
text = " ".join(df_clean["Title"].dropna())
wordcloud = WordCloud(
    width=1000, 
    height=500, 
    background_color="white",
    colormap="magma", # Choose a nice color scheme
    collocations=False # Helps with cleaner word separation
).generate(text)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## ⚙️ Generate and Export the News Digest (HTML File)

> **To create the file `news_digest.html` in your notebook's output, run the cell below.** If you do not wish to generate or download the file, please skip this cell.

Once the cell completes, look for the file in your notebook's file browser/output pane.

In [ ]:
#!/usr/bin/env python
"""
News Digest Generator

This script loads a pretrained Gemma model (using keras_nlp) to analyze the latest AI-related news items.
It extracts key trends, ethical concerns, and future implications, then generates a Markdown-based digest
that includes a word cloud of news titles. The final output is converted into HTML and saved.

Requirements:
- `keras_nlp`, `tensorflow`, `pandas`, `markdown`, `wordcloud`, `matplotlib`
"""

from functools import lru_cache
import re
import keras
import keras_nlp
import tensorflow as tf
from markdown import markdown
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import io
import base64

# -----------------------------------------------------------------------------
# IMPORTANT: Load your datasource and select the relevant columns.
# You must define 'df' (for example via pd.read_csv(...)) before running the script.
#
# Step 4: Select Relevant Columns (using "title", "description", "url", "publishedAt")
df = df[["title", "description", "url", "publishedAt"]]
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# 1. Load the Gemma Model
# -----------------------------------------------------------------------------
precisions = [tf.bfloat16, None]
gemma_lm = None

for precision in precisions:
    try:
        gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(
            "gemma_instruct_2b_en", jit_compile=False, dtype=precision
        )
        print(f"✅ Gemma model loaded successfully with {precision if precision else 'default'} precision.")
        break
    except Exception as e:
        print(f"❌ Error loading Gemma model ({precision}): {e}")

# -----------------------------------------------------------------------------
# 2. Define the AIAnalyzer Class
# -----------------------------------------------------------------------------
class AIAnalyzer:
    """NewsBot: AI-powered analysis of AI-related news."""
    
    def __init__(self, model, output_format="markdown"):
        self.model = model
        self.output_format = output_format

    @lru_cache(maxsize=50)
    def analyze(self, news_item: str) -> dict:
        """
        Generates an AI-powered analysis of a news item.
        Expects news_item to be a string containing the title and description.
        """
        prompt = f"""
Please analyze the following news item and generate an insightful breakdown of it. Do NOT repeat any part of this prompt or include extra instructions in your final response. Base your analysis solely on the information provided.

News item:
{news_item}

Your analysis MUST be exactly structured as follows, starting immediately with "1. Key Trends":
1. Key Trends: Identify industry patterns, emerging technologies, risks, and opportunities evident in this news item.
2. Implications: Discuss real-world applications and ethical concerns related to this news item.
3. AI Insights: Provide an AI-driven perspective on what this news item means for the future.

Ensure your response is well-organized, informative, and engaging.
"""
        try:
            raw = self.model.generate(prompt, max_length=1024).strip()
            response = (
                raw[0] if isinstance(raw, (list, tuple))
                else raw.numpy()[0].decode() if isinstance(raw, tf.Tensor)
                else str(raw)
            ).strip()
        except Exception as e:
            response = f"⚠️ Error generating content: {e}"

        # --- Cleanup Routine ---

        # 1. Trim any text preceding "1. Key Trends:" unconditionally.
        marker_index = response.find("1. Key Trends:")
        if marker_index != -1:
            response = response[marker_index:].strip()

        # 2. Remove any trailing text starting from known prompt ending markers.
        response = re.sub(r'\n\s*Ensure your response.*', '', response, flags=re.IGNORECASE)

        # 3. Remove known unwanted instruction lines line-by-line.
        unwanted_lines = {
            "1. Key Trends:",
            "1. Key Trends: Identify industry patterns, emerging technologies, risks, and opportunities evident in this news item.",
            "2. Implications: Discuss real-world applications and ethical concerns related to this news item.",
            "3. AI Insights: Provide an AI-driven perspective on what this news item means for the future."
        }
        clean_lines = []
        for line in response.splitlines():
            if line.strip() not in unwanted_lines:
                clean_lines.append(line)
        response = "\n".join(clean_lines).strip()

        # 4. Reduce excessive newlines.
        response = re.sub(r"\n{3,}", "\n\n", response).strip()

        # 5. If nothing remains (or if the cleaned response is empty), supply a fallback analysis.
        if not response:
            response = (
                "1. Key Trends: The news item demonstrates significant AI innovation with clear emerging trends that could reshape the industry. "
                "2. Implications: It underscores both opportunities and critical risks, including ethical and practical challenges. "
                "3. AI Insights: Overall, this development signals a transformative evolution in AI technology."
            )

        return {"news_item": news_item, "response": response}

# -----------------------------------------------------------------------------
# 3. Process the News Data
# -----------------------------------------------------------------------------
# Process the top 10 news items from the DataFrame.
news_df = df.head(10)[['title', 'description', 'url', 'publishedAt']]

if gemma_lm is not None:
    analyzer = AIAnalyzer(gemma_lm)
    results = []
    for _, row in news_df.iterrows():
        title = row['title']
        description = row['description']
        # Construct the news item prompt using title and description.
        news_item_prompt = f"Title: {title}\nDescription: {description}"
        analysis_result = analyzer.analyze(news_item_prompt)
        analysis_result.update({
            "title": title,
            "description": description,
            "url": row["url"],
            "publishedAt": row["publishedAt"]
        })
        results.append(analysis_result)
else:
    results = []
    print("No model available; cannot generate news analysis.")

# -----------------------------------------------------------------------------
# 4. Generate Word Cloud & Build Markdown Digest
# -----------------------------------------------------------------------------
text = " ".join(df["title"].dropna())
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)

# Save the word cloud image to a BytesIO object and encode as base64.
buf = io.BytesIO()
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.tight_layout()
plt.savefig(buf, format='png', bbox_inches='tight')
plt.close()

image_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
image_html = f'<img src="data:image/png;base64,{image_base64}" alt="Word Cloud" style="max-width: 100%; height: auto;">'

# -----------------------------------------------------------------------------
# 5. Build Markdown Output and Convert to HTML
# -----------------------------------------------------------------------------
markdown_output = "# 🌟🔎 AI-Powered News Digest 🔥\n\n"
markdown_output += image_html + "\n\n"

for i, rep in enumerate(results, start=1):
    markdown_piece = f"""
---
## {i}. 📰 **{rep['title']}**

**Description:** <br>
{rep['description']}

🕒 **Published At:** {rep['publishedAt']} <br>
🔗 **URL:** [{rep['url']}]({rep['url']}) <br>

{rep['response']}
"""
    markdown_output += markdown_piece

html_content = markdown(markdown_output)

html_page = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>AI-Powered News Digest</title>
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            line-height: 1.6;
            padding: 20px;
            max-width: 900px;
            margin: auto;
            background-color: #f9f9f9; /* Added for a light background */
        }}
        
        h1 {{
            color: #1f4d49;
            text-align: center;
            border-bottom: 2px solid #1f4d49; /* Updated for a stronger header line */
            padding-bottom: 10px;
        }}
        
        h2 {{
            color: #2a647d; /* Changed header color */
            border-bottom: 1px solid #ddd;
            padding-bottom: 5px;
            margin-top: 30px;
        }}
    
        a {{
            color: #007bff; /* Changed link color */
            text-decoration: none;
        }}
        
        a:hover {{
            text-decoration: underline;
        }}
        
        hr {{
            margin: 40px 0;
            border: 0;
            border-top: 5px dotted #e0e0e0; /* Changed to a dotted separator */
        }}
        
        img {{
            display: block;
            margin: 20px auto;
            border: 1px solid #ddd; /* Added border and radius for images */
            border-radius: 5px;
        }}
    </style>
</head>
<body>
{html_content}
</body>
</html>
"""

with open("news_digest.html", "w", encoding="utf-8") as f:
    f.write(html_page)

# 1. Print the success message
print("✅ Page generated successfully! Open 'news_digest.html' in your browser to view your AI-powered news digest.")

# 2. Display the HTML content directly in the notebook output cell
print("\n--- Displaying HTML Output Below ---")
display(HTML(html_page))

<div style="background-color: #F0F8FF; padding: 20px; border-radius: 10px; border: 1px solid #D0E6FA; font-family: Arial, sans-serif; line-height: 1.6; color: #333;">
  
  <h2 style="margin-top: 0;">📰 AI News Summary with Gemma LLM</h2>

  <p>Stay up to date with the most popular <strong>AI</strong> and <strong>LLM</strong> news (excluding anything related to <em>crypto</em>), automatically analyzed using a <strong>Gemma large language model</strong>.</p>

  <hr style="border: none; border-top: 1px solid #D0E6FA; margin: 20px 0;">

  <h3>⚙️ How It Works:</h3>
  <ul style="padding-left: 20px;">
    <li>🔗 <strong>Fetch News:</strong> Pulls articles from NewsAPI using keywords like <code>AI</code> or <code>LLM</code> (excluding <code>crypto</code>).</li>
    <li>🧠 <strong>Analyze with LLM:</strong> Summarizes using the Gemma model.</li>
    <li>📊 <strong>Display Results:</strong> Presents data in a clean, styled table including key metadata.</li>
  </ul>

  <h3>✅ What You Get:</h3>
  <p>A streamlined table featuring the <strong>Top 5 AI-related articles</strong> with:</p>
  <ul style="padding-left: 20px;">
    <li>✍️ <strong>Title</strong></li>
    <li>📰 <strong>LLM-generated Summary</strong></li>
    <li>🏷️ <strong>Source</strong></li>
    <li>📅 <strong>Published Date</strong></li>
    <li>🔗 <strong>Direct Link to Full Article</strong></li>
  </ul>

</div>


In [ ]:
# Constants
NEWS_QUERY = "(AI OR LLM) NOT crypto"
NEWS_SORT = "popularity"
NEWS_LANG = "en"
NEWS_PAGE_SIZE = 10

# Your API key must be set as an environment variable or manually provided
url = (
    f"https://newsapi.org/v2/everything"
    f"?q={NEWS_QUERY}&language={NEWS_LANG}&sortBy={NEWS_SORT}"
    f"&pageSize={NEWS_PAGE_SIZE}&apiKey={NEWS_API_KEY}"
)

# Analysis & Display Settings
LLM_TOP_COUNT = 5
DISPLAY_COUNT = 5

# --- Gemma Analysis Function ---
def analyze_article_with_gemma(title, description):
    """
    Generate a concise summary and sentiment using the Gemma LLM.
    """
    if 'gemma_lm' not in globals() or gemma_lm is None:
        return "LLM Not Loaded: Please initialize 'gemma_lm'."

    prompt = f"""
    Analyze the following news article snippet and generate a summary 
    including its main topic and overall sentiment (positive, negative, or neutral).

    Title: "{title}"
    Snippet: "{description}"
    
    Summary:
    """

    try:
        output = gemma_lm.generate(prompt, max_length=384)
        summary = output.split("Summary:")[-1].strip().split('\n')[0].replace('"', '')
        return summary
    except Exception as e:
        return f"LLM Analysis Error: {e}"

# --- Fetch Articles ---
def fetch_articles():
    try:
        print("📡 Fetching news data...")
        response = requests.get(url)
        response.raise_for_status()
        return response.json().get('articles', [])
    except requests.exceptions.RequestException as e:
        print(f"❌ Request Error: {e}")
        return []

# --- Main Display Logic ---
def process_and_display_articles(articles):
    if not articles:
        print("No articles available to display.")
        return

    print(f"🌟 Displaying Top {DISPLAY_COUNT} AI/LLM Articles with LLM Analysis 🌟\n")

    display_data = []

    for i, article in enumerate(articles[:DISPLAY_COUNT]):
        title = article.get('title', 'N/A')
        description = article.get('description', 'No description available.')
        source = article.get('source', {}).get('name', 'Unknown Source')
        date = article.get('publishedAt', '')[:10] or 'N/A'
        link = article.get('url', '#')

        llm_summary = "N/A"
        if i < LLM_TOP_COUNT:
            print(f"🧠 Analyzing Article {i+1} with Gemma...")
            llm_summary = analyze_article_with_gemma(title, description)

        display_data.append({
            "Rank": i + 1,
            "LLM Analysis": llm_summary,
            "Title": title,
            "Source": source,
            "Date": date,
            "Link": f'<a href="{link}" target="_blank">Read Article</a>'
        })

    df = pd.DataFrame(display_data)

    styled_df = df.style.set_table_styles([
        {'selector': 'th', 'props': [
            ('background-color', '#1E88E5'),
            ('color', '#fff'),
            ('font-size', '14px'),
            ('text-align', 'center'),
            ('padding', '12px 8px'),
            ('border', '1px solid #BBDEFB')
        ]},
        {'selector': 'td', 'props': [
            ('font-family', 'Arial, sans-serif'),
            ('padding', '10px 8px'),
            ('border', '1px solid #E0E0E0'),
            ('vertical-align', 'top')
        ]},
        {'selector': 'tr:nth-child(even)', 'props': [
            ('background-color', '#F9FBFD')
        ]},
        {'selector': 'tr:hover', 'props': [
            ('background-color', '#E3F2FD')
        ]},
        {'selector': 'table', 'props': [
            ('border-collapse', 'collapse'),
            ('width', '100%'),
            ('box-shadow', '0 2px 5px rgba(0, 0, 0, 0.1)')
        ]},
        {'selector': 'a', 'props': [
            ('color', '#1E88E5'),
            ('text-decoration', 'none'),
            ('font-weight', 'bold')
        ]}
    ]).set_properties(
        subset=['Rank', 'Date', 'Link'], **{'text-align': 'center'}
    ).set_properties(
        subset=['LLM Analysis', 'Title'], **{'text-align': 'left', 'max-width': '400px'}
    ).hide(axis='index').set_caption(
        f"📰 Top {DISPLAY_COUNT} AI Articles (Analyzed by Gemma LLM)"
    )

    display(HTML(styled_df.to_html(escape=False)))

# --- Run Pipeline ---
articles = fetch_articles()
process_and_display_articles(articles)

## 🚀 LLM Ethical News Dashboard Pipeline Summary 

This document summarizes the execution of a **three-phase data pipeline** designed to analyze current news trends regarding the ethical implications of Large Language Models (LLMs), culminating in three dynamic, interactive visualizations.

---

### 1. 🐍 Phase 1: Data Acquisition (News API / Python)

* **Action:** Used standard Python libraries (`requests`) to execute a **paginated fetch loop** against the News API, which automatically iterates through all pages to retrieve the **complete, available dataset** over the last **7 days**.
* **Query:** The search used a complex, targeted query focusing on LLMs AND specific ethical topics:
    * **LLM Keywords:** `"large language model" OR LLM OR ChatGPT OR Gemini`
    * **Ethical Focus:** `accountability OR "intellectual property" OR "job displacement" OR "data privacy" OR "environmental impact"`
    * **Exclusions:** Excluded common noise terms like `NOT (bias OR hallucination)`.
* **Result:** A complete list of raw news article dictionaries (`raw_articles`), with a size **dynamically determined** by the API, ready for LLM processing.

---

### 2. 🤖 Phase 2: LLM Data Structuring (GemmaCausalLM)

* **Goal:** To transform raw, unstructured text (titles/descriptions) into clean, categorical data required for plotting.
* **Tool:** The specialized Keras-NLP model, **GemmaCausalLM** (`gemma_lm`), was integrated for classification.
* **Process:** A dedicated Python function runs the `gemma_lm` against each article using a **carefully engineered prompt**. This prompt forces the LLM to perform three core tasks and output a **strict JSON object** for each article:
    * `application`: Classifies the primary LLM industry (e.g., 'Healthcare/Science').
    * `concerns`: Identifies all relevant ethical issues (e.g., \["Data Privacy", "Accountability & IP"]).
    * `entities`: Extracts the key organizations/products (e.g., \["Google", "OpenAI"]).
* **Result:** A clean Pandas DataFrame (`df`) containing the structured, classified data, ready for aggregation.

---

### 3. 📊 Phase 3: Visualization (Pandas & Plotly)

The structured DataFrame was aggregated using **Pandas** and visualized using the **Plotly** library to create three interactive charts that provide comprehensive analysis:

1.  **📈 Line Chart (Trend Over Time):** Shows the **daily volume** of news coverage for each ethical concern category, highlighting trending issues.
2.  **🥧 Pie Chart (Overall Distribution):** Displays the **total percentage share** of each ethical concern across the full dataset.
3.  **📊 Stacked Bar Chart (Entity vs. Application):** Links the **Top 7 Entities** to the specific **Application Areas** they are most frequently associated with in the news.

In [ ]:
# ============================
# 📦 Imports & Setup
# ============================
import os, re, json, math, requests
import pandas as pd
import plotly.express as px
from tqdm import tqdm
from datetime import datetime, timedelta, timezone

import plotly.io as pio
pio.renderers.default = "iframe"   # often the most reliable in Kaggle


# Make sure you set your News API key in Kaggle environment variables
# NEWS_API_KEY = os.getenv("NEWS_API_KEY")
BASE_URL = "https://newsapi.org/v2/everything"


In [ ]:
# ============================
# 🐍 Phase 1: Data Acquisition
# ============================

def build_query():
    must_llm = '("large language model" OR LLM OR ChatGPT OR Gemini)'
    must_ethics = '(accountability OR "intellectual property" OR "job displacement" OR "data privacy" OR "environmental impact")'
    exclusions = 'NOT (bias OR hallucination)'
    return f'{must_llm} AND {must_ethics} AND {exclusions}'

def iso_day_range(days=7):
    now = datetime.now(timezone.utc)
    from_date = (now - timedelta(days=days)).replace(microsecond=0)
    return from_date.isoformat(), now.replace(microsecond=0).isoformat()

def fetch_all_articles(page_size=100, max_pages=None):
    q = build_query()
    from_iso, to_iso = iso_day_range(7)

    params = {
        "q": q,
        "from": from_iso,
        "to": to_iso,
        "language": "en",
        "sortBy": "publishedAt",
        "pageSize": page_size,
        "apiKey": NEWS_API_KEY,
    }

    resp = requests.get(BASE_URL, params=params, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    total = data.get("totalResults", 0)
    pages = math.ceil(total / page_size)
    if max_pages is not None:
        pages = min(pages, max_pages)

    articles = data.get("articles", [])
    for page in tqdm(range(2, pages + 1), desc="Fetching pages"):
        params["page"] = page
        r = requests.get(BASE_URL, params=params, timeout=30)
        r.raise_for_status()
        j = r.json()
        articles.extend(j.get("articles", []))

    seen, raw_articles = set(), []
    for a in articles:
        url = a.get("url")
        if not url or url in seen:
            continue
        seen.add(url)
        raw_articles.append({
            "title": a.get("title") or "",
            "description": a.get("description") or "",
            "source": (a.get("source") or {}).get("name") or "",
            "url": url,
            "publishedAt": a.get("publishedAt") or "",
        })
    return raw_articles

# Example run
# raw_articles = fetch_all_articles(max_pages=2)
# print(len(raw_articles))


In [ ]:
# ============================
# 🤖 Phase 2: Classification
# ============================

CLOSED_APPLICATIONS = {
    "Healthcare/Science","Education","Finance","Government/Public Sector",
    "Enterprise/Work","Consumer","Media/Entertainment","Research","Legal"
}
CLOSED_CONCERNS = {"Data Privacy","Accountability & IP","Job Displacement","Environmental Impact","Other"}

def run_gemma(prompt: str, max_len: int = 512) -> str:
    """Use the already-imported gemma_lm to generate text."""
    output = gemma_lm.generate(prompt, max_length=max_len)
    if isinstance(output, dict) and "text" in output:
        return output["text"]
    elif isinstance(output, list):
        return output[0].get("text", "")
    return str(output)

def extract_json(text: str) -> dict:
    match = re.search(r"\{.*\}", text, flags=re.DOTALL)
    if not match:
        return {}
    try:
        return json.loads(match.group(0))
    except json.JSONDecodeError:
        cleaned = re.sub(r",\s*}", "}", match.group(0))
        cleaned = re.sub(r",\s*]", "]", cleaned)
        try:
            return json.loads(cleaned)
        except:
            return {}

def normalize_fields(obj: dict) -> dict:
    app = obj.get("application", "")
    concerns = obj.get("concerns", [])
    entities = obj.get("entities", [])

    if app not in CLOSED_APPLICATIONS:
        app = "Other"

    if not isinstance(concerns, list):
        concerns = [concerns]
    concerns = [c for c in concerns if c in CLOSED_CONCERNS] or ["Other"]

    if not isinstance(entities, list):
        entities = [entities]
    entities = [e for e in entities if isinstance(e, str) and e.strip()] or ["Other"]

    return {"application": app, "concerns": concerns, "entities": entities}

def classify_articles(raw_articles):
    rows = []
    for a in tqdm(raw_articles, desc="Classifying"):
        prompt = f"""
You are classifying news articles about the ethical implications of Large Language Models.

Return ONLY a valid JSON object with keys: application, concerns, entities.

Rules:
- application: choose ONE from {list(CLOSED_APPLICATIONS)}
- concerns: choose ALL that apply from {list(CLOSED_CONCERNS)}
- entities: list key organizations/products mentioned explicitly. If unknown, use "Other".

TITLE: {a['title']}
DESCRIPTION: {a['description']}
"""
        out = run_gemma(prompt)
        obj = extract_json(out)
        norm = normalize_fields(obj)
        rows.append({
            "title": a["title"],
            "description": a["description"],
            "source": a["source"],
            "url": a["url"],
            "publishedAt": a["publishedAt"],
            "application": norm["application"],
            "concerns": norm["concerns"],
            "entities": norm["entities"],
        })
    df = pd.DataFrame(rows)
    df = df.explode("concerns").reset_index(drop=True)
    df["date"] = pd.to_datetime(df["publishedAt"]).dt.date
    return df


In [ ]:
# ============================
# 📊 Phase 3: Visualization
# ============================

def plot_trend_over_time(df):
    trend = (df.groupby(["date", "concerns"])
               .size()
               .reset_index(name="count"))
    fig = px.line(
        trend,
        x="date",
        y="count",
        color="concerns",
        markers=True,
        title="Daily news volume by ethical concern"
    )
    fig.show()

def plot_overall_distribution(df):
    dist = df.groupby("concerns").size().reset_index(name="count")
    fig = px.pie(
        dist,
        names="concerns",
        values="count",
        title="Overall distribution of ethical concerns"
    )
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.show()

def plot_entities_vs_application(df, top_n=7):
    df_ent = df.explode("entities")
    top_entities = df_ent["entities"].value_counts().head(top_n).index.tolist()
    filtered = df_ent[df_ent["entities"].isin(top_entities)]
    pivot = (filtered.groupby(["entities", "application"])
             .size()
             .reset_index(name="count"))
    fig = px.bar(
        pivot,
        x="entities",
        y="count",
        color="application",
        title=f"Top {top_n} entities by associated application areas",
        barmode="stack"
    )
    fig.show()


In [ ]:
# ============================
# 🚀 Run the Full Pipeline
# ============================

# 1. Acquire
raw_articles = fetch_all_articles(max_pages=2)  # limit pages for demo
print(f"Fetched {len(raw_articles)} articles")

# 2. Classify
df = classify_articles(raw_articles)
print(df.head())

# 3. Visualize
plot_trend_over_time(df)
plot_overall_distribution(df)
plot_entities_vs_application(df)


📊 Generate and display the charts

In [ ]:
# 1. Line chart: daily volume per ethical concern
trend = df.groupby(["date", "concerns"]).size().reset_index(name="count")
fig1 = px.line(trend, x="date", y="count", color="concerns", markers=True,
               title="📈 Daily News Volume by Ethical Concern")
fig1.show()

In [ ]:
# 2. Pie chart: overall distribution of ethical concerns
dist = df["concerns"].value_counts().reset_index()
dist.columns = ["concerns", "count"]
fig2 = px.pie(dist, names="concerns", values="count",
              title="🥧 Overall Distribution of Ethical Concerns")
fig2.update_traces(textposition='inside', textinfo='percent+label')
fig2.show()

In [ ]:
# 3. Stacked bar chart: top entities vs application areas
df_exp = df.explode("entities")
top_entities = df_exp["entities"].value_counts().head(7).index.tolist()
filtered = df_exp[df_exp["entities"].isin(top_entities)]
pivot = filtered.groupby(["entities", "application"]).size().reset_index(name="count")

fig3 = px.bar(pivot, x="entities", y="count", color="application",
              title="📊 Top Entities vs Application Areas", barmode="stack")
fig3.show()

# 📰 Automated News Analysis: From Headlines to Actionable Insights

## 🚀 Introduction: Uncovering Trends with NewsAPI and Sentiment Analysis

Welcome to this core section of the automated news analysis project! The goal here is to transform raw news headlines into **actionable intelligence** on any given topic.

---

### **Pipeline Overview**

1.  **Dynamic Data Retrieval (NewsAPI):**
    * We use the **NewsAPI** with Python to dynamically fetch a high-volume stream of the latest global news articles related to a specified keyword (e.g., 'Artificial Intelligence', 'Climate Policy', 'Market Volatility').

2.  **Quantifying the Mood (VADER Sentiment Analysis):**
    * The VADER (Valence Aware Dictionary and sEntiment Reasoner) library is applied to the headline and article text. VADER is specifically attuned to social media and general text, making it highly effective for news analysis.
    * It quantifies the prevailing media mood, assigning a **Compound Sentiment Score** (ranging from -1.0 for extremely negative to +1.0 for extremely positive) to each article.

---

### **🎯 Deliverables: The Power of the Output**

The combination of data retrieval and sentiment scoring generates highly readable outputs:

* **A Comprehensive DataFrame:** A clean, organized table showing the news source, headline, publish date, and the calculated sentiment scores (Positive, Negative, Neutral, and Compound).
* **Intuitive Visualization:** A chart illustrating the sentiment distribution over time and across news sources.

This snapshot of media coverage helps us quickly identify:

| Insight | Description |
| :--- | :--- |
| **Emerging Trends** | Spikes in coverage (volume) combined with highly positive sentiment. |
| **Risk Areas** | Consistent, high-volume coverage with pronounced negative sentiment. |
| **Market Excitement** | Sudden, strong positive sentiment correlated with a key event. |

In [ ]:
# 1. Installation and Imports
!pip install newsapi-python vaderSentiment
import pandas as pd
from newsapi import NewsApiClient
from kaggle_secrets import UserSecretsClient
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# --- 💡 USER INPUT CONFIGURATION ---
# Define the search term here.
search_query = "AI or artificial intelligence"
# Define the desired number of articles to fetch per page (max 100 on free tier)
PAGE_SIZE = 100
# Define the language (e.g., 'en', 'de', 'fr')
LANGUAGE = 'en'
# ------------------------------------

print(f"Configuration set: Analyzing news for '{search_query}'...")

# 2. Secure API Client Initialization
print("Initializing NewsAPI Client...")
try:
    secret_client = UserSecretsClient()
    api_key = secret_client.get_secret("NEWS_API_KEY")
    newsapi = NewsApiClient(api_key=api_key)
    print("API Client successfully initialized.")
except Exception as e:
    print(f"Error accessing secret: {e}. Please ensure 'NEWS_API_KEY' is set in Kaggle Secrets.")
    # Use placeholder to allow subsequent cells to run, but with an empty article list
    articles = []
    newsapi = None

In [ ]:
# 3. Fetch News Articles
articles = []
if newsapi:
    print(f"Fetching articles for query: '{search_query}'...")
    try:
        all_articles = newsapi.get_everything(
            q=search_query,
            language=LANGUAGE,
            sort_by='relevancy',
            page_size=PAGE_SIZE
        )
        articles = all_articles['articles']
        total_results = all_articles['totalResults']
        print(f"Successfully retrieved {len(articles)} articles (Total found: {total_results}).")
    except Exception as e:
        print(f"An error occurred during the API call: {e}")
        articles = []

if not articles:
    print("\n⚠️ No articles retrieved. Check your API key and query.")

In [ ]:
# 4. Data Processing and DataFrame Creation
if articles:
    news_df = pd.DataFrame(articles)
    
    # Extract source name
    news_df['source_name'] = news_df['source'].apply(lambda x: x['name'] if isinstance(x, dict) and 'name' in x else 'Unknown')
    news_df.drop('source', axis=1, inplace=True) 
    
    # Convert date to datetime object
    news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt'], utc=True)
    
    # Select and rename relevant columns
    news_df = news_df[['publishedAt', 'source_name', 'title', 'description', 'url', 'content']]
    news_df.rename(columns={'publishedAt': 'Date', 'source_name': 'Source', 'title': 'Title', 'description': 'Description', 'url': 'URL'}, inplace=True)
    
    print("DataFrame successfully created and cleaned.")
    print(f"DataFrame Shape: {news_df.shape}")
else:
    print("Skipping DataFrame creation as no articles were found.")

In [ ]:
# 5. Sentiment Analysis (VADER)
if 'news_df' in locals() and not news_df.empty:
    analyzer = SentimentIntensityAnalyzer()
    
    def get_sentiment_score(text):
        """Analyzes sentiment for a given text and returns the VADER compound score."""
        if pd.isna(text) or text is None or str(text) == 'None':
            return 0.0
        # Use description for sentiment
        return analyzer.polarity_scores(str(text))['compound']

    def categorize_sentiment(score):
        """Categorizes the VADER compound score."""
        if score >= 0.05:
            return 'Positive'
        elif score <= -0.05:
            return 'Negative'
        else:
            return 'Neutral'

    # Apply sentiment analysis
    news_df['Sentiment_Score'] = news_df['Description'].apply(get_sentiment_score)
    news_df['Sentiment'] = news_df['Sentiment_Score'].apply(categorize_sentiment)
    
    print("Sentiment analysis complete.")
else:
    print("Skipping sentiment analysis.")

In [ ]:
# 6. Styled Data Preview for Readability

if 'news_df' in locals() and not news_df.empty:
    
    # Define a helper function to color sentiment scores
    def color_sentiment(val):
        """Applies a color gradient based on the sentiment score."""
        if val >= 0.05:
            # Green for Positive
            color = '#90EE90' 
        elif val <= -0.05:
            # Light Red for Negative
            color = '#F08080' 
        else:
            # Neutral/Grey
            color = '#D3D3D3'
        return f'background-color: {color}'

    # Prepare DataFrame for display: show only key columns
    display_cols = ['Date', 'Source', 'Title', 'Sentiment', 'Sentiment_Score']
    styled_df = news_df[display_cols].head(10).style \
        .set_caption(f"Top 10 News Articles for: '{search_query.title()}'") \
        .applymap(color_sentiment, subset=['Sentiment_Score']) \
        .background_gradient(cmap='viridis', subset=['Sentiment_Score']) \
        .format({'Date': lambda t: t.strftime('%Y-%m-%d %H:%M'), 'Sentiment_Score': "{:.3f}"}) \
        .set_properties(**{'font-size': '10pt', 'border-color': 'lightgrey'}) \
        .hide(axis='index') # Hides the default index column
        
    display(styled_df)
else:
    print("Cannot display data: DataFrame is empty.")

In [ ]:
# 7. Visualization: Sentiment Distribution

if 'news_df' in locals() and not news_df.empty:
    plt.figure(figsize=(8, 5))
    
    # Define colors for the chart
    color_map = {'Positive': '#4CAF50', 'Neutral': '#FFC107', 'Negative': '#F44336'}
    
    sns.countplot(x='Sentiment', 
                  data=news_df, 
                  order=['Positive', 'Neutral', 'Negative'], 
                  palette=color_map)
    
    plt.title(f'Sentiment Distribution for News on: "{search_query.title()}"', 
              fontsize=14, 
              fontweight='bold')
    plt.xlabel('Sentiment Category', fontsize=12)
    plt.ylabel('Number of Articles', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()
    
    # Summary box
    positive_count = news_df[news_df['Sentiment'] == 'Positive'].shape[0]
    total_count = news_df.shape[0]
    positive_percent = (positive_count / total_count) * 100 if total_count > 0 else 0
    
    display(HTML(f"""
    <div style="border: 1px solid #007BFF; padding: 10px; border-radius: 5px; background-color: #E6F0FF;">
        <p><strong>📊 Analysis Summary:</strong></p>
        <ul>
            <li>**Total Articles:** {total_count}</li>
            <li>**Positive Articles:** {positive_count} ({positive_percent:.1f}%)</li>
            <li>**Most Frequent Source:** {news_df['Source'].mode()[0] if not news_df.empty else 'N/A'}</li>
        </ul>
    </div>
    """))

else:
    print("Skipping visualization: DataFrame is empty.")